Scrape images from www.shufadict.com/dict/x

<table>
    <tr>
        <td> <img src="./../data/shufadict/raw/对/0e0c5c1e77602807bdead70d6bb24391.png" alt="sample1" style="height: 80px"> </td>
        <td> <img src="./../data/shufadict/raw/就/0e5a9ed779e6337c62948e3992404fce.png" alt="sample2" style="height: 80px"> </td>
        <td> <img src="./../data/shufadict/raw/成/9b5e9f4126d0bac2def6f3a375c90719.png" alt="sample3" style="height: 80px"> </td>
        <td> <img src="./../data/shufadict/raw/然/1b74a2eea6138f3791590d6189348e4d.png" alt="sample4" style="height: 80px"> </td>
    </tr>
</table>

In [ ]:
%cd ..

import os
import json
import yaml
import time
import random
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import urllib
from urllib.error import HTTPError

In [ ]:
CHAR_LS_PATH = 'data/common.txt'
DIR_RAW = 'data/shufadict/raw'
CHROME_PATH = '/Users/kx/docs/chromedriver'

In [ ]:
# get the 100 common character
common = []
with open(CHAR_LS_PATH, 'r') as f:
    for line in f:
        common.append(line.strip())

common = common[:100]

In [ ]:
# send query character to text input field
def send_char_key(char, driver):
    elem = driver.find_element_by_class_name('MuiInputBase-input')
    
    # clear input field
    while len(elem.get_attribute('value')) > 0:
        elem.send_keys(Keys.BACK_SPACE)

    elem.send_keys(char)
    
    # click send cutton
    elem = driver.find_element_by_class_name("MuiButton-label")
    script = "arguments[0].click()"
    driver.execute_script(script, elem)
    
    assert char == driver.find_element_by_class_name('MuiInputBase-input').get_attribute('value')
    

# extract all image elements
def get_img_wrappers(driver):
    content = driver.page_source
    soup = BeautifulSoup(content, features='lxml')
    wrappers = soup.find_all(
        'div', 
        {
            'class': 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-4 MuiGrid-grid-sm-3 MuiGrid-grid-md-2 MuiGrid-grid-lg-2'
        }
    )
    imgs = []
    for wrapper in wrappers:
        imgs.append((wrapper.find('img'), wrapper.find('p').text))
    return imgs

In [ ]:
# launch headless chrome driver
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(CHROME_PATH, options=chrome_options)

# go to shufadict home page
dict_url = 'https://www.shufadict.com/dict/x'
driver.get(dict_url)

# wait for text box to show up
loading = True
while loading:
    content = driver.page_source
    soup = BeautifulSoup(content)
    loading = soup.find('div', {'id': 'loading'}) is not None
    time.sleep(3)

In [ ]:
prev_img = ''
for i, char in enumerate(common):

    print('[STATUS] {}/{} {}'.format(i + 1, len(common), char))

    send_char_key(char, driver) # load images for the given char

    char_path = os.path.join(DIR_RAW, char)
    if not os.path.exists(char_path):
        os.makedirs(char_path)
    
    retries = 3
    imgs = get_img_wrappers(driver)
    
    # sanity check to ensure the page got reloaded, and limit max retries to 3
    while (len(imgs) == 0 or imgs[0] == prev_img) and retries > 0:
        time.sleep(random.random() + 5)
        imgs = get_img_wrappers(driver)
        retries -= 1
    
    # sanity check
    if len(imgs) == 0:
        print('[ERROR] 0 images found'.format(char))
    
    # start downloading images
    for item in tqdm(imgs):
    
        (img, author) = item
        img_src = img['src']
        filename = img_src.split('/')[-1].split('@')[0]
        img_path = os.path.join(char_path, filename)
        
        if not os.path.exists(img_path):
            
            try:
                # download image
                response = requests.get(img_src)
                file = open(img_path, "wb")
                file.write(response.content)
                file.close()
                # urllib.request.urlretrieve(img_src, img_path) # alternative way to download
                
            except HTTPError:
                print('[ERROR] HTTPError -', img_src)
    
    prev_img = imgs[0]

In [ ]:
driver.quit()